### Profile report

import pandas_profiling

df= pd.read_csv("RSC_reviews_with_category.csv",index_col=0)
report=df.profile_report(title='Reviewbox SalesRank Data')
report.to_file(output_file ='SalesRank_report.html')

In [1]:
#Import neccessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Word Count Comparison 

### Objective: comparing the word count of the individual products with the word count of this category level
### Outline
1. filter columns
2. create total_word metrics
3. cut word bins with ranges
4. Aggregate across products
5. Merge product and category
6. Merge by joining it on category_id



In [2]:
# Filter column of interest
df= pd.read_csv("RSC_reviews_with_category.csv",index_col=0)
sales = pd.read_csv("Sales.csv")
df = df[['product','text','category_id2']]

#Let's create the word count column
df['totalwords'] = df['text'].str.split().str.len()

#Create word bins with appropriate ranges
df['word_bins'] = pd.cut(x=df['totalwords'], bins=[0, 5, 15, 25, 40, 65, 100, 200, 100000])
df['word_bins'] = pd.cut(x=df['totalwords'], bins=[0, 5, 15, 25, 40, 65, 100, 200, 100000], labels=['0 - 5 words', '6 - 15 words', '16 - 25 words', '26 - 40 words', '41 - 65 words', '66 - 100 words', '101 - 200 words','200+'])

# Create a dataframe to aggregate word bins across products & categories
# Normalize to get proportions
product_aggregation = pd.crosstab(df["product"], df["word_bins"], margins=True, normalize='index')
category_aggregation = pd.crosstab(df["category_id2"], df["word_bins"], margins=True, normalize='index')

# Merge the two features table together
product_aggregation = pd.merge(product_aggregation,sales, how = 'inner', left_on = "product", right_on="id")
product_aggregation = pd.merge(product_aggregation,category_aggregation, how = 'inner', left_on = "category_id2", right_on="category_id2")

#product_aggregation.to_csv("Word_count_features.csv")

In [3]:
product_aggregation

,0 - 5 words_x,6 - 15 words_x,16 - 25 words_x,26 - 40 words_x,41 - 65 words_x,66 - 100 words_x,101 - 200 words_x,200+_x,Unnamed: 0,id,category_id2,0 - 5 words_y,6 - 15 words_y,16 - 25 words_y,26 - 40 words_y,41 - 65 words_y,66 - 100 words_y,101 - 200 words_y,200+_y
0,0.181818,0.272727,0.181818,0.272727,0.000000,0.000000,0.090909,0.000000,1275677,B0000AVVPU,pet-supplies/2975425011,0.135165,0.174176,0.134066,0.143956,0.157692,0.092308,0.101099,0.061538
1,0.411765,0.176471,0.235294,0.117647,0.058824,0.000000,0.000000,0.000000,1198750,B0000DAPGK,pet-supplies/2975425011,0.135165,0.174176,0.134066,0.143956,0.157692,0.092308,0.101099,0.061538
2,0.087324,0.135211,0.135211,0.163380,0.205634,0.101408,0.126761,0.045070,1634447,B00062F6HE,pet-supplies/2975425011,0.135165,0.174176,0.134066,0.143956,0.157692,0.092308,0.101099,0.061538
3,0.000000,0.000000,0.571429,0.000000,0.285714,0.142857,0.000000,0.000000,1612857,B00062F6OM,pet-supplies/2975425011,0.135165,0.174176,0.134066,0.143956,0.157692,0.092308,0.101099,0.061538
4,0.173077,0.298077,0.153846,0.173077,0.125000,0.038462,0.038462,0.000000,1514430,B00068R98C,pet-supplies/2975425011,0.135165,0.174176,0.134066,0.143956,0.157692,0.092308,0.101099,0.061538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,0.046332,0.135135,0.127413,0.162162,0.123552,0.135135,0.193050,0.077220,1442888,B071213DDY,pet-supplies/17592594011,0.046332,0.135135,0.127413,0.162162,0.123552,0.135135,0.193050,0.077220
937,0.162162,0.297297,0.216216,0.162162,0.000000,0.108108,0.054054,0.000000,1251609,B071JTYQVC,pet-supplies/3024230011,0.049333,0.150000,0.122000,0.159333,0.206667,0.156667,0.126000,0.030000
938,0.046480,0.146275,0.119617,0.159262,0.211893,0.157895,0.127820,0.030759,1444041,B0721735K9,pet-supplies/3024230011,0.049333,0.150000,0.122000,0.159333,0.206667,0.156667,0.126000,0.030000
939,0.054878,0.073171,0.097561,0.146341,0.152439,0.182927,0.146341,0.146341,16588,B0756GW8SJ,diy,0.054878,0.073171,0.097561,0.146341,0.152439,0.182927,0.146341,0.146341


## Rating Distribution / Review Count history

### Objective: find number of reviews and their ratings over 1/3/5 months interval
### Outline
1. Clean data transformation
2. create total_word metrics
3. cut word bins with ranges
4. Aggregate across products
5. Merge product and category
6. Merge by joining it on category_id






### 6 Features:
* (2) change in proportion/delta of 1 star reviews compared to last quarter
* (2) change in proportion/delta of 5 star reviews compared to last quarter
* (1) change in count of 1 star reviews compared to last quarter
* (1) change in number of reviews


### Other methods:

In [6]:
from dfply import *
df= pd.read_csv("RSC_reviews_with_category.csv",index_col=0)

df['date']=pd.to_datetime(df['date'])

df['year']=df['date'].dt.year
df['week']=df['date'].dt.day
df['month']=df['date'].dt.month
df=df >> mask(df.week<30, df.week>=6)

#Create week bins with appropriate ranges
df['week_group'] = pd.cut(x=df['week'], bins=[0,6, 18, 30], labels=['Q1','Q2','Q3'])

In [5]:
sub=df[['product','stars','year','week_group']]
sub=sub >> mask(sub.year==2019)
sub_1=sub[sub.stars==1]
sub_5=sub[sub.stars==5]

In [9]:
sub_1.groupby(['week_group','product']).size()

week_group  product   
Q1          B000241NRI     1
            B00063446M     2
            B00068R98C     1
            B00074L4RW     2
            B00074L4UO     2
                          ..
Q3          B075RYQ35W    18
            B0762TKBL2     5
            B076XDFBFR     1
            B0775219GQ     1
            B078N83GS4     1
Length: 437, dtype: int64

In [10]:
## 1 star review || 5 star review count
count_five_star = pd.crosstab(sub_5["product"], sub_5["week_group"])
count_one_star = pd.crosstab(sub_1["product"], sub_1["week_group"])

In [11]:
## # review count
count_review= pd.crosstab(sub["product"], sub["week_group"])
count_review['Q2']-count_review['Q1']

In [23]:
count_five_star=count_five_star.rename(columns={'Q1':'Q1_count_5star','Q2':'Q2_count_5star','Q3':'Q3_count_5star'})
count_one_star=count_one_star.rename(columns={'Q1':'Q1_count_1star','Q2':'Q2_count_1star','Q3':'Q3_count_1star'})
count_review=count_review.rename(columns={'Q1':'Q1_count_review','Q2':'Q2_count_review','Q3':'Q3_count_review'})

In [15]:
#count_five_star['Q2']-count_five_star['Q1']
#count_five_star['Q3']-count_five_star['Q2']


## Rating Distribution / Review Count history

### Product Stars Trend
* average stars for past three months
* num of varified purchase 


### Product Trend

In [2]:
df = pd.read_csv("RSC_reviews_with_category.csv",index_col=0)
df['date'] = pd.to_datetime(df['date'])

In [3]:
df['verified']= df['verified'].map({True: 1, False: 0})

In [4]:
# num product count
product_count=df.groupby('product')['source'].agg("count")
# avg stars
avg_stars=df.groupby('product')['stars'].agg("mean")

#### Date related Functions

In [15]:
max_date=df['date'].max()
df['date_since_review']=(max_date-df['date']).apply(lambda x: x.days)
def last_n_month(df,n,col='date_since_review',stars=None):
    if(stars is not None):
        df=df[df['stars']==stars]
    v=n*30
    new_df=df[df[col]<=v].reset_index(drop=True)
    return(new_df)
        
def get_n_quarter(df,n,col='date_since_review',stars=None):
    if(stars is not None):
        df=df[df['stars']==stars]
    # Q1
    v=n*90
    v2=(n-1)*90
    new_df=df[(df[col]<=v) & (df[col]>=v2) ].reset_index(drop=True)
    return(new_df)

In [16]:
agg_last_month=last_n_month(df,1,stars=5).groupby('product')
agg_last_3month=last_n_month(df,3,stars=5).groupby('product')
#agg_last_year=last_n_month(df,12,stars=5).groupby('product')
star_last_month=agg_last_month['stars'].mean()
star_last_3month=agg_last_3month['stars'].mean()

verified_last_month=agg_last_month['verified'].mean()
verified_last_3month=agg_last_3month['verified'].mean()

In [79]:
parameter = 'stars'
stars_review = df.groupby('product')[['stars']].count()
star_df = df[['product','stars']]
stars_count = pd.crosstab(star_df['product'], star_df['stars'])
stars_count["5/1 ratio"]=stars_count[5]/stars_count[1]



### Exploration

In [65]:
## Verified reviews
pd.crosstab(df['product'], df['verified'], dropna=True).head(5)

verified,0,1
product,,
B0000AVVPU,0,11
B0000BYCM0,0,65
B0000DAPGK,0,17
B0001ZWZ9S,0,19
B00023N7TG,4,233


In [62]:
# Stars distribution
pd.crosstab(df['product'], df['stars']).head(5)

stars,1,2,3,4,5
product,,,,,
B0000AVVPU,0,1,1,3,6
B0000BYCM0,10,0,6,11,38
B0000DAPGK,2,0,0,2,13
B0001ZWZ9S,3,3,3,0,10
B00023N7TG,15,5,16,40,161
